In [1]:
from pyspark import SparkContext

# spark context
sc = SparkContext("local", "FIM")


In [2]:
# Read file
file =  sc.textFile("fruits.txt")

print(file.collect())

['Apple,Mango,Banana', 'Banana,Mango', 'Apple,Banana', 'Apple,Mango,Coconut', 'Strawberry,Grapes,Lemon,Raspberry', 'Rassberry,Grapes', 'Strawberry,Apple', 'Apple,Mango,Raspberry', 'Mango,Raspberry', 'Mango,Apple', 'Apple,Raspberry', 'Banana,Raspberry,Mango', 'Apple,Mango,Banana', 'Raspberry,Banana', 'Apple,Strawberry', 'Strawberry,Banana,Apple,Mango', 'Mango,Banana,Raspberry,Apple', 'Coconut,Apple,Raspberry', 'Raspberry,Coconut,Banana']


In [3]:
## Splited items  
lblitems = file.map(lambda line: line.split(','))

print(lblitems.collect())

[['Apple', 'Mango', 'Banana'], ['Banana', 'Mango'], ['Apple', 'Banana'], ['Apple', 'Mango', 'Coconut'], ['Strawberry', 'Grapes', 'Lemon', 'Raspberry'], ['Rassberry', 'Grapes'], ['Strawberry', 'Apple'], ['Apple', 'Mango', 'Raspberry'], ['Mango', 'Raspberry'], ['Mango', 'Apple'], ['Apple', 'Raspberry'], ['Banana', 'Raspberry', 'Mango'], ['Apple', 'Mango', 'Banana'], ['Raspberry', 'Banana'], ['Apple', 'Strawberry'], ['Strawberry', 'Banana', 'Apple', 'Mango'], ['Mango', 'Banana', 'Raspberry', 'Apple'], ['Coconut', 'Apple', 'Raspberry'], ['Raspberry', 'Coconut', 'Banana']]


In [4]:
## Whole lines in single array 
wlitems = file.flatMap(lambda line:line.split(','))

print(wlitems.collect())

['Apple', 'Mango', 'Banana', 'Banana', 'Mango', 'Apple', 'Banana', 'Apple', 'Mango', 'Coconut', 'Strawberry', 'Grapes', 'Lemon', 'Raspberry', 'Rassberry', 'Grapes', 'Strawberry', 'Apple', 'Apple', 'Mango', 'Raspberry', 'Mango', 'Raspberry', 'Mango', 'Apple', 'Apple', 'Raspberry', 'Banana', 'Raspberry', 'Mango', 'Apple', 'Mango', 'Banana', 'Raspberry', 'Banana', 'Apple', 'Strawberry', 'Strawberry', 'Banana', 'Apple', 'Mango', 'Mango', 'Banana', 'Raspberry', 'Apple', 'Coconut', 'Apple', 'Raspberry', 'Raspberry', 'Coconut', 'Banana']


In [5]:
uniqueItems = wlitems.distinct()

# Add 1 as Tuple
supportRdd = wlitems.map(lambda item: (item , 1))

# Method for sum in reduceByKey method
def sumOparator(x,y):
    return x+y

# Sum of values by key
supportRdd = supportRdd.reduceByKey(sumOparator)

In [6]:
supports = supportRdd.map(lambda item: item[1]) # Return only support values


In [7]:
# Define minimum support value 
minSupport = supports.min()

# If mininmum support is 1 then replace it with 2 
minSupport = 2 if minSupport == 1 else minSupport

## Filter first supportRdd with minimum support 
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

## Craete base RDD with will be updated every iteration
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
print('1 . Table has crated...') 

supportRdd = supportRdd.map(lambda item: item[0])
supportRddCart = supportRdd

1 . Table has crated...


In [8]:

def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [9]:
c = 2 # Combination length 

while(supportRdd.isEmpty() == False):

    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(sumOparator)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
    c = c+1 

2 . Table has crated... 
3 . Table has crated... 
4 . Table has crated... 


In [10]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

        
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

# Create Filter Object
ff = Filter()

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

  
print(baseRddConfidence.collect())

# : Aggregated support values preparing for the confidence calculatations
# : Aggregated support values are ready !
[[['Apple'], ['Mango'], 58.333333333333336], [['Mango'], ['Apple'], 70.0], [['Apple'], ['Banana'], 41.66666666666667], [['Apple'], ['Coconut'], 16.666666666666664], [['Banana'], ['Apple'], 55.55555555555556], [['Coconut'], ['Apple'], 66.66666666666666], [['Apple'], ['Strawberry'], 25.0], [['Apple'], ['Raspberry'], 33.33333333333333], [['Mango'], ['Banana'], 60.0], [['Banana'], ['Mango'], 66.66666666666666], [['Mango'], ['Raspberry'], 40.0], [['Strawberry'], ['Apple'], 75.0], [['Raspberry'], ['Apple'], 44.44444444444444], [['Raspberry'], ['Mango'], 44.44444444444444], [['Banana'], ['Raspberry'], 44.44444444444444], [['Raspberry'], ['Banana'], 44.44444444444444], [['Coconut'], ['Raspberry'], 66.66666666666666], [['Raspberry'], ['Coconut'], 22.22222222222222], [['Apple'], ['Mango', 'Banana'], 33.33333333333333], [['Mango'], ['Apple', 'Banana'], 40.0], [['Banana'], ['Mango', 

In [13]:
## Import pandas modules
import pandas as pd

## Create an array with collected baseRddConfidence results
result = baseRddConfidence.collect()

## Create Data Frame
confidenceTable = pd.DataFrame(data = result , columns=["Before", "After" , "Confidence"])

# sort confidencetable on confidence coloumn
#confidenceTable = confidenceTable.sort_values(by=['Confidence'], ascending=False, inplace=True)
## Show data frame
print(confidenceTable)

                 Before                After  Confidence
0               [Apple]              [Mango]   58.333333
1               [Mango]              [Apple]   70.000000
2               [Apple]             [Banana]   41.666667
3               [Apple]            [Coconut]   16.666667
4              [Banana]              [Apple]   55.555556
5             [Coconut]              [Apple]   66.666667
6               [Apple]         [Strawberry]   25.000000
7               [Apple]          [Raspberry]   33.333333
8               [Mango]             [Banana]   60.000000
9              [Banana]              [Mango]   66.666667
10              [Mango]          [Raspberry]   40.000000
11         [Strawberry]              [Apple]   75.000000
12          [Raspberry]              [Apple]   44.444444
13          [Raspberry]              [Mango]   44.444444
14             [Banana]          [Raspberry]   44.444444
15          [Raspberry]             [Banana]   44.444444
16            [Coconut]        